In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df_train = pd.read_csv('data/train.csv', sep=',', encoding='utf8')

In [3]:
df_train.shape

(24712, 22)

In [4]:
df_test = pd.read_csv('data/test.csv', sep=',', encoding='utf8')

In [5]:
df_test.shape

(16476, 21)

In [6]:
y = df_train['target'].values
y

array([0, 0, 0, ..., 1, 0, 0])

In [7]:
df_train = df_train.drop('target', axis=1)

In [8]:
#df_train.info()

In [9]:

def preproc_data(df_input):
    df_output = df_input.copy()
    df_output = df_output.drop(['contact'], axis=1)
    df_output = df_output.drop(['_id'], axis=1)
    list=[]
    for i in df_output: 
        if str(df_output[i].dtype) == 'object': 
            list.append(i)
            
    df_output = pd.get_dummies(df_output, columns=list)
    return df_output


In [10]:
train = df_train.pipe(preproc_data)

In [11]:
#train.info()

In [12]:
train.head()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,...,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success
0,54,87,1,999,0,1.4,93.444,-36.1,4.963,5228.1,...,0,0,0,0,0,1,0,0,1,0
1,36,291,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,0,0,0,1,0,0,0,0,1,0
2,53,182,1,999,0,1.4,93.444,-36.1,4.965,5228.1,...,0,0,0,1,0,0,0,0,1,0
3,34,180,2,999,0,1.1,93.994,-36.4,4.857,5191.0,...,0,0,0,0,0,0,1,0,1,0
4,29,6,1,999,0,-1.7,94.027,-38.3,0.890,4991.6,...,0,0,1,0,0,0,0,0,1,0


In [13]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score, precision_score, recall_score
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.33, random_state=42)

In [14]:
#-------------------------- KNN GridsearchCV, cross_validation, roc_auc_score ---------------------

In [15]:
from sklearn.grid_search import GridSearchCV
n_neighbors_array = [50, 75, 90, 100, 150]
knn = KNeighborsClassifier()
grid = GridSearchCV(knn, param_grid = {'n_neighbors': n_neighbors_array})
grid.fit(X_train, y_train)
best_cv_err = 1 - grid.best_score_
best_n_neighbors = grid.best_estimator_.n_neighbors
print(best_cv_err, best_n_neighbors)#8.7% на ошибку

/Users/mikhail/anaconda3/lib/python3.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/mikhail/anaconda3/lib/python3.7/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


0.08703267500150991 100


In [16]:
knn = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier(n_neighbors=100))
])

In [17]:
knn.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('knn', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=100, p=2,
           weights='uniform'))])

In [18]:
knn_pred = knn.predict(X_test)

In [19]:
knn_proba = knn.predict_proba(X_test)

In [20]:
from sklearn.cross_validation import cross_val_score
kn = KNeighborsClassifier(n_neighbors=100)
cross_val_score(
    kn,
    train,
    y,
    scoring='roc_auc',
    cv=10
)


array([0.92572163, 0.92745163, 0.93442805, 0.94025059, 0.92513079,
       0.9160494 , 0.94477147, 0.93348524, 0.92343479, 0.92965181])

In [21]:
roc_auc_score(y_test, knn_proba[:, 1])

0.8855061119210497

In [22]:
#----------------------------------------- decision tree train

In [23]:
from scipy.stats import randint as randint
from scipy.stats import uniform

try:
    from sklearn.model_selection import RandomizedSearchCV
    from sklearn.model_selection import StratifiedKFold
except ImportError:
    from sklearn.cross_validation import RandomizedSearchCV
    from sklearn.cross_validation import StratifiedKFold
RND_SEED = 123

In [24]:
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': randint(2, 10),
    'min_samples_leaf': randint(5, 12),
    'class_weight': [None, 'balanced']}

cv = StratifiedKFold(n_splits=5, random_state=123, shuffle=True)

model = DecisionTreeClassifier(random_state=123)
random_search = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=300, n_jobs=-1,
                                   cv=cv, scoring='roc_auc', random_state=123)
random_search.fit(X_train, y_train)



RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=123, shuffle=True),
          error_score='raise',
          estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=123,
            splitter='best'),
          fit_params=None, iid=True, n_iter=300, n_jobs=-1,
          param_distributions={'criterion': ['gini', 'entropy'], 'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x1a1688a6a0>, 'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x1a1688a7f0>, 'class_weight': [None, 'balanced']},
          pre_dispatch='2*n_jobs', random_state=123, refit=True,
          return_train_score='warn', scoring='roc_auc', verbose=0)

In [25]:
random_search.best_params_

{'class_weight': 'balanced',
 'criterion': 'entropy',
 'max_depth': 6,
 'min_samples_leaf': 11}

In [26]:
random_search.best_score_

0.9309749384309101

In [27]:
dtc = DecisionTreeClassifier(criterion='entropy', max_depth=5,class_weight='balanced', min_samples_leaf=7, random_state=3)
cross_val_score(
    dtc,
    train,
    y,
    scoring='roc_auc',
    cv=10
)

array([0.93244716, 0.92822879, 0.92910809, 0.942779  , 0.93604365,
       0.92815599, 0.94677125, 0.93570308, 0.92920791, 0.93425242])

In [28]:
Tree = DecisionTreeClassifier(criterion='entropy', max_depth=5,class_weight='balanced', min_samples_leaf=7, random_state=3)

Tree.fit(X_train, y_train)
predictTree = Tree.predict(X_test)
predict_probaTree = Tree.predict_proba(X_test)
predict_probaTree


array([[0.08590351, 0.91409649],
       [1.        , 0.        ],
       [1.        , 0.        ],
       ...,
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.56588713, 0.43411287]])

In [29]:
roc_auc_score(y_test, predict_probaTree[:, 1])

0.9312797278606408

In [30]:
#-------------------------- logistic train

In [31]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.cross_validation import KFold
fold = KFold(len(y_train), n_folds=5, shuffle=True, random_state=777)

In [32]:
"""l = LogisticRegression(penalty='l2', random_state=777, max_iter=10000,solver='newton-cg', C=1)
cross_val_score(
    l,
    train,
    y,
    scoring='roc_auc',
    cv=10
)"""


"l = LogisticRegression(penalty='l2', random_state=777, max_iter=10000,solver='newton-cg', C=1)\ncross_val_score(\n    l,\n    train,\n    y,\n    scoring='roc_auc',\n    cv=10\n)"

In [33]:
Logistic = LogisticRegression(penalty='l2', random_state=777, max_iter=10000,solver='newton-cg', C=1)
Logistic.fit(X_train, y_train)
predictLogistic= Logistic.predict(X_test)
predict_probaLogistic = Logistic.predict_proba(X_test)


/Users/mikhail/anaconda3/lib/python3.7/site-packages/scipy/optimize/linesearch.py:462: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/mikhail/anaconda3/lib/python3.7/site-packages/scipy/optimize/linesearch.py:313: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/mikhail/anaconda3/lib/python3.7/site-packages/sklearn/utils/optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')


In [34]:
predict_probaLogistic

array([[0.37790828, 0.62209172],
       [0.99361822, 0.00638178],
       [0.99292373, 0.00707627],
       ...,
       [0.98953056, 0.01046944],
       [0.98749581, 0.01250419],
       [0.92823014, 0.07176986]])

In [35]:
roc_auc_score(y_test, predict_probaLogistic[:, 1])

0.9343291839557399

In [36]:
MetricFrame = pd.DataFrame([[precision_score(y_test, predictTree), 
                            recall_score(y_test, predictTree), accuracy_score(y_test, predictTree)]
                           ,[precision_score(y_test, knn_pred), 
                            recall_score(y_test, knn_pred), accuracy_score(y_test, knn_pred)],
                            [precision_score(y_test, predictLogistic), 
                            recall_score(y_test, predictLogistic), accuracy_score(y_test, predictLogistic)]
                           ],
                          columns=["precision","recall","accuracy"]
                          )

MetricFrame

,precision,recall,accuracy
0,0.391041,0.905946,0.829307
1,0.673077,0.189189,0.897609
2,0.670139,0.417297,0.910607


In [37]:
#--------------------------------------------------------- test submission

In [38]:
test = df_test.pipe(preproc_data)

In [39]:

model = Logistic.predict(test)
predictions = Logistic.predict_proba(test)


In [40]:
pd.value_counts(model) 

0    15314
1     1162
dtype: int64

In [41]:
predictions[73,1]


0.020709751466711657

In [42]:
submission = pd.DataFrame({'_id':df_test['_id'],'target':predictions[:,1]})
submission.head()

,_id,target
0,66810d8e6bf2b41c880a7bc6c8a1e295,0.056417
1,ccac3879652b08cb8b44c1920fd93afa,0.013207
2,fcccab4d7a76f70647f015f2c84c2af8,0.014567
3,ed8399278c30678dab739045fa12b440,0.043173
4,1d4d62ac5cabcb48bac7112813f290cb,0.015305


In [43]:
submission2 = pd.DataFrame({'_id':df_test['_id'],'target':model})
submission2.head()

,_id,target
0,66810d8e6bf2b41c880a7bc6c8a1e295,0
1,ccac3879652b08cb8b44c1920fd93afa,0
2,fcccab4d7a76f70647f015f2c84c2af8,0
3,ed8399278c30678dab739045fa12b440,0
4,1d4d62ac5cabcb48bac7112813f290cb,0


In [44]:
#filename = 'targets.csv'
#submission.to_csv(filename,index=False)
#print('Saved file: ' + filename)

In [46]:
filename = 'data/team5result-elena.csv'
submission2.to_csv(filename,index=False)
print('Saved file: ' + filename)

Saved file: data/team5result-elena.csv
